In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Nichole\.cache\kagglehub\datasets\blastchar\telco-customer-churn\versions\1


In [9]:
import pandas as pd
import os

file_path = os.path.join(path, "WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = pd.read_csv(file_path)
print(df.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [ ]:
# Quantidade de clientes que cancelaram o serviço e que não cancelaram
churn_counts = df['Churn'].value_counts()
print("Churn counts:\n", churn_counts)

# porcentagem de clientes que cancelaram o serviço
churn_percentage = df['Churn'].value_counts(normalize=True) * 100
print("Churn percentage:\n", churn_percentage.round(2))

Churn counts:
 Churn
No     5174
Yes    1869
Name: count, dtype: int64
Churn percentage:
 Churn
No     73.46
Yes    26.54
Name: proportion, dtype: float64


In [43]:
# O objetivo é analisar os possíveis motivos que levam um cliente a cancelar o serviço identificando padrões de comportamento entre os clientes que permanecem e os que cancelam.

# Primeiro, vou criar duas variáveis separadas: uma para os clientes que cancelaram o serviço e outra para os que não cancelaram.
churned_customers = df[df['Churn'] == 'Yes']

# Somar a quantidade de clientes que cancelaram o serviço
sum_churned = churned_customers.shape[0]
print("Total de clientes que cancelaram o serviço:", sum_churned)

retained_customers = df[df['Churn'] == 'No']
# Somar a quantidade de clientes que não cancelaram o serviço
sum_retained = retained_customers.shape[0]
print("Total de clientes que não cancelaram o serviço:", sum_retained)

print("Churned Customers Shape:", churned_customers.head())

print("Retained Customers Shape:", retained_customers.head())

Total de clientes que cancelaram o serviço: 1869
Total de clientes que não cancelaram o serviço: 5174
Churned Customers Shape:     customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
2   3668-QPYBK    Male              0      No         No       2          Yes   
4   9237-HQITU  Female              0      No         No       2          Yes   
5   9305-CDSKC  Female              0      No         No       8          Yes   
8   7892-POOKP  Female              0     Yes         No      28          Yes   
13  0280-XJGEX    Male              0      No         No      49          Yes   

   MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
2             No             DSL            Yes  ...               No   
4             No     Fiber optic             No  ...               No   
5            Yes     Fiber optic             No  ...              Yes   
8            Yes     Fiber optic             No  ...              Yes   
13           Yes     

In [ ]:
# Agora, vou analisar separadamente as características de cada grupo para identificar diferenças significativas que possam indicar os motivos do cancelamento.
# Por exemplo, a coluna "tenure" (tempo de permanência) pode ser um fator importante. Vou calcular a média do tempo de permanência para ambos os grupos.
avg_tenure_churned = churned_customers['tenure'].mean()
print("Average Tenure of Churned Customers:", avg_tenure_churned.round(2))
avg_tenure_retained = retained_customers['tenure'].mean()
print("Average Tenure of Retained Customers:", avg_tenure_retained.round(2))

# O resultado foi que os clientes que cancelaram o serviço têm, em média, um tempo de permanência significativamente menor do que os clientes que permaneceram.
# Isso sugere que o tempo de permanência pode ser um fator importante no cancelamento do serviço.

# 🤨 Mas por qual motivo os clientes novatos cancelaram o contrato? O que poderia fazer com que eles fiquem mais tempo na empresa?

Average Tenure of Churned Customers: 17.98
Average Tenure of Retained Customers: 37.57


In [ ]:
# Qual gênero mais cancela o serviço?

sum_churned_women = churned_customers[churned_customers['gender']
                                      == 'Female'].shape[0]
print("Total de mulheres que cancelaram o serviço:", sum_churned_women)

sum_churned_man = churned_customers[churned_customers['gender']
                                    == 'Male'].shape[0]
print("Total de homens que cancelaram o serviço:", sum_churned_man)

# O resultado não parece interferir muito, mas as mulheres cancelaram um pouco mais o serviço do que os homens.
# É necessário analisar cada coluna dentro do dataset para entender quais fatores influenciam no cancelamento do serviço.
# Fonte de livro:
# “Practical Statistics for Data Scientists”, O’Reilly (Bruce et al., 2020), cap. 2–3
# → Apresenta a necessidade de análises univariadas estratificadas por classe como primeira etapa exploratória.

Total de mulheres que cancelaram o serviço: 939
Total de homens que cancelaram o serviço: 930


In [ ]:
# Agora vou analisar a coluna "SeniorCitizen" para ver se há alguma diferença entre os clientes idosos e não idosos em relação ao cancelamento do serviço.

sum_churned_senior = churned_customers[churned_customers['SeniorCitizen'] == 1].shape[0]
print("Total de idosos que cancelaram o serviço:", sum_churned_senior)
sum_churned_not_senior = churned_customers[churned_customers['SeniorCitizen'] == 0].shape[0]
print("Total de não idosos que cancelaram o serviço:", sum_churned_not_senior)

# O resultado mostra que os clientes não idosos cancelaram o serviço mais vezes do que os idosos.
# Mas isso não diz nada.

# Vou descobrir quantos idosos e não idosos existem no dataset para entender melhor esses números.
total_senior = df[df['SeniorCitizen'] == 1].shape[0]
print("Total de idosos no dataset:", total_senior)
total_not_senior = df[df['SeniorCitizen'] == 0].shape[0]
print("Total de não idosos no dataset:", total_not_senior)

# Cálculo de taxas de cancelamento dos idosos e não idosos
churn_rate_senior = (sum_churned_senior / total_senior) * 100
print("Taxa de cancelamento dos idosos: {:.2f}%".format(churn_rate_senior))
churn_rate_not_senior = (sum_churned_not_senior / total_not_senior) * 100
print("Taxa de cancelamento dos não idosos: {:.2f}%".format(
    churn_rate_not_senior))

# Podemos ver que os idosos têm uma taxa de cancelamento maior do que os não idosos.
# Isso sugere que ser idoso pode ser um fator que influencia no cancelamento do serviço
# Relative Risk (RR) Risk Ratio razão de riscos

rr = churn_rate_senior / churn_rate_not_senior
print(
    "Relative Risk (RR) de idosos em relação aos não idosos: {:.2f}".format(rr))

Total de idosos que cancelaram o serviço: 476
Total de não idosos que cancelaram o serviço: 1393
Total de idosos no dataset: 1142
Total de não idosos no dataset: 5901
Taxa de cancelamento dos idosos: 41.68%
Taxa de cancelamento dos não idosos: 23.61%
Relative Risk (RR) de idosos em relação aos não idosos: 1.77


In [ ]:
# Agora vou analisar a coluna 'Partner' para ver se há alguma diferença entre os clientes que têm parceiro e os que não têm em relação ao cancelamento do serviço.

import pandas as pd
import scipy
from scipy.stats import chi2_contingency

sum_churned_partner = churned_customers[churned_customers['Partner']
                                        == 'Yes'].shape[0]
print("Total de clientes com parceiro que cancelaram o serviço:", sum_churned_partner)
sum_churned_no_partner = churned_customers[churned_customers['Partner']
                                           == 'No'].shape[0]
print("Total de clientes sem parceiro que cancelaram o serviço:",
      sum_churned_no_partner)

# Vou descobrir quantos clientes com parceiro e sem parceiro existem no dataset para entender melhor esses números.
total_partner = df[df['Partner'] == 'Yes'].shape[0]
total_no_partner = df[df['Partner'] == 'No'].shape[0]
print("Total de clientes com parceiro:", total_partner)
print("Total de clientes sem parceiro:", total_no_partner)

# Cálculo de taxas de cancelamento dos clientes com parceiro e sem parceiro
churn_rate_partner = (sum_churned_partner / total_partner) * 100
churn_rate_no_partner = (sum_churned_no_partner / total_no_partner) * 100
print("Taxa de cancelamento dos clientes com parceiro:",
      churn_rate_partner)
print("Taxa de cancelamento dos clientes sem parceiro:",
      churn_rate_no_partner)

# É possível ver que os clientes sem parceiro têm uma taxa de cancelamento maior do que os clientes com parceiro.
# Isso sugere que não ter um parceiro pode ser um fator que influencia no cancelamento do serviço.
# Relative Risk (RR) Risk Ratio razão de riscos
rr_partner = churn_rate_no_partner / churn_rate_partner
print("Relative Risk (RR) de clientes sem parceiro em relação aos com parceiro: {:.2f}".format(
    rr_partner))


# Tabela de contingência
data = [
    [669, 2733],   # Partner = Yes
    [1200, 2441]   # Partner = No
]

# Rodando o teste qui-quadrado
chi2, p_value, dof, expected = chi2_contingency(data)

print("Chi-square:", chi2)
print("p-valor:", p_value)
print("Graus de liberdade:", dof)
print("Valores esperados:", expected)

# Com essas interpretações de  p-valor e chi-square, podemos concluir que há uma associação significativa entre ter um parceiro e o cancelamento do serviço.
# No caso, clientes sem parceiro têm uma taxa de cancelamento maior do que clientes com parceiro.

Total de clientes com parceiro que cancelaram o serviço: 669
Total de clientes sem parceiro que cancelaram o serviço: 1200
Total de clientes com parceiro: 3402
Total de clientes sem parceiro: 3641
Taxa de cancelamento dos clientes com parceiro: 19.66490299823633
Taxa de cancelamento dos clientes sem parceiro: 32.95797857731392
Relative Risk (RR) de clientes sem parceiro em relação aos com parceiro: 1.68
Chi-square: 158.7333820309922
p-valor: 2.1399113440759935e-36
Graus de liberdade: 1
Valores esperados: [[ 902.78830044 2499.21169956]
 [ 966.21169956 2674.78830044]]


In [ ]:
# Agora, vou fazer a mesma análise para a coluna 'Dependents', que indica se o cliente tem dependentes ou não.
sum_churned_dependents = churned_customers[churned_customers['Dependents']
                                           == 'Yes'].shape[0]
print("Total de clientes com dependentes que cancelaram o serviço:",
      sum_churned_dependents)
sum_churned_no_dependents = churned_customers[churned_customers['Dependents']
                                              == 'No'].shape[0]
print("Total de clientes sem dependentes que cancelaram o serviço:",
      sum_churned_no_dependents)

# Taxa de cancelamento dos clientes com dependentes e sem dependentes
total_dependents = df[df['Dependents'] == 'Yes'].shape[0]
total_no_dependents = df[df['Dependents'] == 'No'].shape[0]
print("Total de clientes com dependentes:", total_dependents)
print("Total de clientes sem dependentes:", total_no_dependents)

# Cálculo de taxas de cancelamento dos clientes com dependentes e sem dependentes
churn_rate_dependents = (sum_churned_dependents / total_dependents) * 100
churn_rate_no_dependents = (
    sum_churned_no_dependents / total_no_dependents) * 100
print("Taxa de cancelamento dos clientes com dependentes:", churn_rate_dependents)
print("Taxa de cancelamento dos clientes sem dependentes:", churn_rate_no_dependents)

Total de clientes com dependentes que cancelaram o serviço: 326
Total de clientes sem dependentes que cancelaram o serviço: 1543
Total de clientes com dependentes: 2110
Total de clientes sem dependentes: 4933
Taxa de cancelamento dos clientes com dependentes: 15.450236966824646
Taxa de cancelamento dos clientes sem dependentes: 31.279140482465028
